In [61]:
# Import our dependencies
import pandas as pd
from path import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [155]:
# Import our clean dataset
file_path = Path("../clean_data/combined_olympic_data.csv")
df = pd.read_csv(file_path)
df.head(10)

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,total
0,Afghanistan,33370794,613.856689,0.465,0.676,12,1
1,Albania,2889104,4578.631994,0.733,0.267,33,0
2,United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0
3,Argentina,42669500,12334.798250,0.836,0.364,34,4
4,Armenia,2912403,3986.231624,0.733,0.299,37,2
5,Australia,23475686,62510.791170,0.935,0.123,80,38
6,Austria,8546356,51717.495940,0.885,0.083,72,17
7,Azerbaijan,9535079,7891.313147,0.751,0.330,29,9
8,Burundi,9844297,274.857948,0.400,0.483,20,0
9,Belgium,11209057,47700.540360,0.890,0.076,76,3


In [156]:
df["total"].value_counts()

0      75
1      18
2       8
3       6
4       5
6       5
8       4
17      3
5       3
9       3
12      3
13      3
7       2
15      2
18      2
38      2
100     1
19      1
16      1
21      1
23      1
31      1
36      1
43      1
44      1
46      1
50      1
63      1
70      1
97      1
132     1
Name: total, dtype: int64

In [157]:
bin_labels = ['0', '1-5', '6-10', '11-30', '31-50', '51+']
num_bins = len(bin_labels)

df['medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 30, 50, 140],
                              labels=bin_labels)
df['numerical_medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 30, 50, 140],
                              labels=range(num_bins))

In [158]:
df['medal_grouping'].value_counts()

0        75
1-5      40
11-30    17
6-10     14
31-50     8
51+       5
Name: medal_grouping, dtype: int64

In [159]:
new_df = df.rename(columns={"total": "count_of_medals"})
new_df.head()

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping
0,Afghanistan,33370794,613.856689,0.465,0.676,12,1,1-5,1
1,Albania,2889104,4578.631994,0.733,0.267,33,0,0,0
2,United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0,0,0
3,Argentina,42669500,12334.798250,0.836,0.364,34,4,1-5,1
4,Armenia,2912403,3986.231624,0.733,0.299,37,2,1-5,1


### Split our preprocessed data into our features and target arrays

In [160]:
# Create our target
y = new_df["numerical_medal_grouping"].values
y[:20]

[1, 0, 0, 1, 1, ..., 1, 0, 3, 0, 3]
Length: 20
Categories (6, int64): [0 < 1 < 2 < 3 < 4 < 5]

In [161]:
# Optimizing and transforming features
# Transforming CPI to values between 0-1
new_df["corruption_perceptions_index"] = new_df["corruption_perceptions_index"] / 100
# Transforming GII to invert values. Higher values will now correspond to more gender equality.
new_df["gender_inequality_index"] = 1 - new_df["gender_inequality_index"]
# Replacing GDP per capita with total GDP
new_df["gdp_total"] = new_df["population"] * new_df["gdp_per_capita"]
new_df.head()

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping,gdp_total
0,Afghanistan,33370794,613.856689,0.465,0.324,0.12,1,1-5,1,2.048489e+10
1,Albania,2889104,4578.631994,0.733,0.733,0.33,0,0,0,1.322814e+10
2,United Arab Emirates,9214175,43751.838890,0.835,0.753,0.70,0,0,0,4.031371e+11
3,Argentina,42669500,12334.798250,0.836,0.636,0.34,4,1-5,1,5.263197e+11
4,Armenia,2912403,3986.231624,0.733,0.701,0.37,2,1-5,1,1.160951e+10


In [162]:
# Create our features
X = new_df.drop(["count_of_medals", "medal_grouping", "numerical_medal_grouping"], axis=1)
X

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,gdp_total
0,Afghanistan,33370794,613.856689,0.465,0.324,0.12,2.048489e+10
1,Albania,2889104,4578.631994,0.733,0.733,0.33,1.322814e+10
2,United Arab Emirates,9214175,43751.838890,0.835,0.753,0.70,4.031371e+11
3,Argentina,42669500,12334.798250,0.836,0.636,0.34,5.263197e+11
4,Armenia,2912403,3986.231624,0.733,0.701,0.37,1.160951e+10
...,...,...,...,...,...,...,...
154,Kosovo,1812771,4080.330717,0.733,0.733,0.33,7.396705e+09
155,Yemen,25823485,1673.146354,0.498,0.243,0.19,4.320647e+10
156,South Africa,54545991,6433.187277,0.666,0.600,0.44,3.509046e+11
157,Zambia,15399753,1763.057298,0.586,0.459,0.38,2.715065e+10


In [163]:
# Splitting data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=15)
X_train.shape
X_test.shape
y_train.shape
y_test.shape
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape, new_df.shape)

(119, 7) (40, 7) (119,) (40,) (159, 10)


In [164]:
#Create a StandardScaler instances
scaler = MinMaxScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train.drop(["country_name"], axis = 1))

# Scale the data
X_train_scaled = X_scaler.transform(X_train.drop(["country_name"], axis = 1))
X_test_scaled = X_scaler.transform(X_test.drop(["country_name"], axis = 1))


In [165]:
X_test

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,gdp_total
157,Zambia,15399753,1763.057298,0.586,0.459,0.38,2.715065e+10
63,Iran,77465753,5585.526758,0.766,0.486,0.27,4.326870e+11
40,Algeria,38923687,5493.025590,0.736,0.571,0.36,2.138088e+11
31,Congo Republic,4736974,2996.918026,0.591,0.406,0.23,1.419632e+10
117,Portugal,10401062,22074.300760,0.830,0.897,0.63,2.295962e+11
74,Cambodia,15274503,1093.496191,0.555,0.518,0.21,1.670261e+10
130,Sao Tome and Principe,195727,1782.798059,0.555,0.473,0.42,3.489417e+08
142,Trinidad and Tobago,1362342,20270.859370,0.772,0.659,0.38,2.761584e+10
111,Peru,30090359,6672.880255,0.734,0.611,0.38,2.007894e+11
25,Switzerland,8188649,86605.563380,0.930,0.955,0.86,7.091826e+11


In [166]:
# Saving values for result comparison later on
y_test_values = y_test

In [167]:
# We need to convert our target labels (expected values) to categorical data
num_classes = num_bins
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# Original label of `1` is one-hot encoded as `0100000000`
y_train[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [168]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train.drop(["country_name"],axis=1).columns)
hidden_nodes_layer1 = 3 * number_input_features
hidden_nodes_layer2 = 2 * number_input_features
hidden_nodes_layer3 = 2 * number_input_features

nn = Sequential()
number_input_features

6

In [169]:
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=num_classes, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 18)                126       
_________________________________________________________________
dense_33 (Dense)             (None, 12)                228       
_________________________________________________________________
dense_34 (Dense)             (None, 12)                156       
_________________________________________________________________
dense_35 (Dense)             (None, 6)                 78        
Total params: 588
Trainable params: 588
Non-trainable params: 0
_________________________________________________________________


In [170]:
# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [146]:
# Train the model
X_train_scaled.reshape(-1,1)
y_train.reshape(-1,1)
fit_model = nn.fit(X_train_scaled, y_train, epochs=150, shuffle=True)

Train on 119 samples
Epoch 1/150
 32/119 [=======>......................] - ETA: 3s

InvalidArgumentError:  Incompatible shapes: [32,6] vs. [32]
	 [[node metrics/accuracy/Equal (defined at /opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_20471]

Function call stack:
distributed_function


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Make predictions with scaled test data
y_test_pred = nn.predict(X_test_scaled)

In [147]:
y_test

array([[[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       ...,

       [[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.

In [148]:
y_test_pred.round(1)

array([[0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.3, 0.5, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.2, 0. , 0. ],
       [0.7, 0.2, 0. , 0. , 0. , 0. ],
       [0.9, 0.1, 0. , 0. , 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.1, 0.1, 0.1, 0.1, 0.3, 0.2],
       [0.2, 0.3, 0.2, 0.2, 0.1, 0. ],
       [0.5, 0.3, 0.1, 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.2, 0. , 0. ],
       [0.5, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.7, 0.3, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.6, 0.3, 0.1, 0. , 0. , 0. ],
       [0.2, 0.2, 0.2, 0.2, 0.1, 0. ],
       [0. , 0. , 0.1, 0.1, 0.4, 0.5],
       [0.2, 0.2, 0.2, 0.2, 0.1, 0. ],
       [0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.2, 0. , 0. ],
       [0.6, 0.3, 0.1, 0. , 0. , 0. ],
       [0.4, 0.4, 0.1, 0.

In [149]:
nn.predict_classes(X_test_scaled)

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [154]:
results_df = pd.DataFrame(y_test_values)
y_pred = nn.predict_classes(X_test_scaled)
# results_df['model_results'] = nn.predict_classes(X_test_scaled)
# results_df = nn.predict_classes(X_test_scaled)
# results_df.head(50)

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0


### Saving the model

In [151]:
# # Export our model to HDF5 file
# nn.save("nn_trained_categories_model_.h5")

In [138]:
# new_df.reset_index(inplace=True)
new_results_df["country_name"] =  X_test["country_name"]
new_results_df["predictions"] = y_pred
new_results_df

# new_results_df = new_df.join(results_df).head(50)
# pd.concat([new_df, results_df], axis=1)

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


RecursionError: maximum recursion depth exceeded

In [27]:
new_results_df = new_results_df.dropna()

In [28]:
new_results_df.drop(["population","gdp_per_capita","human_development_index","gender_inequality_index","corruption_perceptions_index","gdp_total",], axis=1)

# new_results_df = new_results_df[["country_name",0,"model_results"]]

,country_name,count_of_medals,medal_grouping,numerical_medal_grouping,0,model_results
0,Afghanistan,1,1-5,1,0,0.0
1,Albania,0,0,0,3,1.0
2,United Arab Emirates,0,0,0,1,1.0
3,Argentina,4,1-5,1,0,0.0
4,Armenia,2,1-5,1,1,1.0
5,Australia,38,31-50,4,0,0.0
6,Austria,17,11-30,3,0,0.0
7,Azerbaijan,9,6-10,2,1,1.0
8,Burundi,0,0,0,0,1.0
9,Belgium,3,1-5,1,3,4.0


In [28]:
new_results_df.set_index("country_name",drop=True)

,0,model_results
country_name,,
Afghanistan,0,0.0
Albania,3,0.0
United Arab Emirates,1,0.0
Argentina,0,0.0
Armenia,1,1.0
Australia,0,0.0
Austria,0,0.0
Azerbaijan,1,1.0
Burundi,0,0.0


In [29]:
# new_results_df.to_csv("nn_model_results.csv")